In [5]:
import pandas as pd
import yfinance as yf
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Download or Load Tesla Data

# Option 1: Download Tesla data from Yahoo Finance (you can adjust the date range)
tesla_data = yf.download('TSLA', start='2023-01-01', end='2024-01-01')

# Option 2: Load Tesla data from CSV file (uncomment and adjust the path if using a CSV)
# tesla_data = pd.read_csv('tesla_stock_data.csv')

# Display the first few rows of the data
print(tesla_data.head())

# Step 2: Preprocess the Data

# 2.1: Handle missing values (forward fill)
tesla_data = tesla_data.fillna(method='ffill')

# 2.2: Use the 'Close' price for predictions (or create other features as needed)
tesla_data['Close'] = tesla_data['Close'].astype(float)

# 2.3: Normalize the 'Close' price using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
tesla_data['Close_scaled'] = scaler.fit_transform(tesla_data[['Close']])

# Display the preprocessed data
print(tesla_data[['Close', 'Close_scaled']].head())

# Step 3: Prepare Data for EGRU Model

# Parameters for sequence length and input size
seq_len = 10  # Use past 10 days to predict the next day's price
input_size = 1  # We're using 'Close_scaled', which is a single feature

# Create sequences of data
X = []
y = []
for i in range(seq_len, len(tesla_data)):
    X.append(tesla_data['Close_scaled'].iloc[i-seq_len:i].values)  # Past 10 days
    y.append(tesla_data['Close_scaled'].iloc[i])  # Next day's price

X = np.array(X)
y = np.array(y)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Reshape y to (batch_size, 1)

# Check the shape of the prepared data
print(X_tensor.shape)  # Should output (batch_size, seq_len, input_size)
print(y_tensor.shape)  # Should output (batch_size, 1)

# Step 4: Create DataLoader for batching

# Create a TensorDataset and DataLoader for batching
batch_size = 64
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Step 5: Define the EGRU Model (with PyTorch)

class EGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(EGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Pass through the GRU layer
        out, _ = self.gru(x)
        # Only take the output of the last time step
        out = out[:, -1, :]
        # Pass through the fully connected layer
        out = self.fc(out)
        return out

# Step 6: Initialize and Train the EGRU Model

# Hyperparameters for the model
input_size = 1  # Single feature (Close price)
hidden_size = 50  # Number of hidden units in the GRU
output_size = 1  # We want to predict a single value (next day's price)
learning_rate = 0.001
num_epochs = 100

# Initialize the model
model = EGRU(input_size, hidden_size, output_size)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 7: Evaluate the Model

# Put the model in evaluation mode
model.eval()

# Example: Predict for the last sequence in the dataset
with torch.no_grad():
    test_input = X_tensor[-1:].unsqueeze(0)  # Take the last sequence, add batch dimension
    prediction = model(test_input)
    predicted_price = scaler.inverse_transform(prediction.numpy())  # Convert back to original scale
    print(f'Predicted next day price: {predicted_price[0][0]:.2f}')

# Optional: Plot the predictions (for visualization purposes)
import matplotlib.pyplot as plt

predicted_prices = []
for i in range(len(X_tensor)):
    with torch.no_grad():
        pred = model(X_tensor[i:i+1])
        predicted_prices.append(pred.item())

# Inverse transform predicted prices back to original scale
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# Plot actual vs predicted prices
plt.plot(tesla_data['Date'][seq_len:], tesla_data['Close'][seq_len:], label='Actual Prices')
plt.plot(tesla_data['Date'][seq_len:], predicted_prices, label='Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title('Tesla Stock Price Prediction')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_7016/3921303528.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tesla_data = tesla_data.fillna(method='ffill')


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-01-03  118.470001  118.800003  104.639999  108.099998  108.099998   
2023-01-04  109.110001  114.589996  107.519997  113.639999  113.639999   
2023-01-05  110.510002  111.750000  107.160004  110.339996  110.339996   
2023-01-06  103.000000  114.389999  101.809998  113.059998  113.059998   
2023-01-09  118.959999  123.519997  117.110001  119.769997  119.769997   

               Volume  
Date                   
2023-01-03  231402800  
2023-01-04  180389000  
2023-01-05  157986300  
2023-01-06  220911100  
2023-01-09  190284000  
                 Close  Close_scaled
Date                                
2023-01-03  108.099998      0.000000
2023-01-04  113.639999      0.029907
2023-01-05  110.339996      0.012092
2023-01-06  113.059998      0.026776
2023-01-09  119.769997      0.062999
torch.Size([240, 10])
torch.Size([240, 1])


RuntimeError: input.size(-1) must be equal to input_size. Expected 1, got 10